# Reinforcement Learning: Q-Learning

*Prepared by Damian Dailisan*

---

## Problem: `Taxi-v3`

This task was introduced in [Dietterich2000] to illustrate some issues in hierarchical reinforcement learning. There are 4 locations (labeled by different letters) and your job is to pick up the passenger at one location and drop him off in another. You receive +20 points for a successful dropoff, and lose 1 point for every timestep it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.

[Dietterich2000] T Erez, Y Tassa, E Todorov, "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition", 2011.

[OpenAI Gym](https://gym.openai.com/envs/Taxi-v3/) provides a convenient environment for this scenario for us to focus on the actual problem at hand.
It even has a way to render a representation of the problem!

In [1]:
import gym

env = gym.make("Taxi-v3").env

env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


### States

Taxi states $\mathbf{s}$ for this environment involve four variables:
* taxi row location (0,1,2,3,4)
* taxi column location (0,1,2,3,4)
* passenger position (R,G,Y,B,in_taxi)
* destination index (R,G,Y,B)

All in all, there are a total of $5\times 5\times 5\times 4=500$ possible states in the entire system.
As a convenience, the environment provides an [encoding](https://github.com/openai/gym/blob/91d278f2dd7fa4493ad12ad5cba9018726415f3c/gym/envs/toy_text/taxi.py#L184) which converts each possible combination of state variables $s_i$ into a single number from 0-499.

In [2]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Actions and Rewards

In reinforcement learning, it is important to also define the actions $a$ that can be taken from the current state.
To facilitate learning, each corresponding state-action pair must also have a corresponding reward $r$ which provides a heuristic feedback measure on how well the agent is achieving is task.

An agent in `taxi-v3` can perform 6 actions: 
- 0: move south
- 1: move north
- 2: move east
- 3: move west
- 4: pickup passenger
- 5: drop off passenger

And there are three types of rewards possible:
- -1 per step reward unless other reward is triggered.
- +20 delivering passenger.
- -10  executing "pickup" and "drop-off" actions illegally.


### Transitions
`taxi-v3` also maintains a lookup table of states, actions, and the corresponding transitions and rewards in `env.P`.
This transition table provides the probability of state $s'$ from ($s$,$a$), along with the corresponding probability of each outcome, rewards, and boolean variable indicating if the environment should terminate.
Particularly for `taxi-v3`, there is only a single possible outcome state $s'$ for each state-action pair.
In this sense, actions in `taxi-v3` are *deterministic*.
Other environments and actions might call for *stochastic* actions; where a state-action pair may result to multiple possible states.

In [3]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

## Solving `Taxi-v3` without RL

Let us *solve* the `Taxi-v3` environment without using RL.
To do this, we can use the default transition matrix defined by the environment.
We can sample actions given the current state (note: actions are equally likely) and perform the chosen action to transition to the next state and incurring the corresponding rewards.

In [4]:
env.s = 328  # set environment to illustration's state

steps = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    steps += 1
    
    
print("Timesteps taken: {}".format(steps))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 3239
Penalties incurred: 1061


We can see that the agent takes a long time to accomplish its task (transporting a passenger from the origin to destination).
Along the way, the agent also incurs a lot of penalties for attempting to pick up a non-existing passenger, or dropping off the passenger at the wrong location.

Even if we re-run this several times, we are likely to get similarly bad results because there are no updates to the **probabilities of actions** --- the agent is not learning!

### Animation

This is an animation of our naive agent.

In [5]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames, dt=0.1):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(dt)
        
print_frames(frames, dt=0.05)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 3239
State: 0
Action: 5
Reward: 20


## Q learning

To make our agent learn from its past experiences in the environment, we need a way for the agent to quantitatively compare different action outcomes.
One such approach is to define an *state-action value function*, which we denote $Q(s,a)$ (Q here can be thought of as *quality*).
$$ Q(s,a)= \mathbb{E}(r + \gamma \max_{a} Q(s',a)) $$
The $Q$-value estimates the expected cumulative reward given the current state and action.
The above equation is also known as a Bellman equation.
The Bellman equation tells us that the maximum future reward is the reward the agent received for entering the current state $s$ plus the maximum future reward for the next state $s'$.

Now that the agent has a way to quantify the consequences of its actions, we can define the driving force of the agent: given its current state, it wants to pick the action that provides the maximum $Q$-value.
However, as with all machine learning, it is also important for an agent to once in a while explore other actions.
Typically, this is implemented using an $\epsilon$-greedy (epsilon-greedy) approach: with a probability $\epsilon$, the agent will take a random action.

We can then let the agent interact repeatedly with the environment to experience the different states and rewards possible.
While interactive with the environment, the agent then *learns* by updating the table of $Q$-values for all possible states using the following rule:
$$ Q_{t+1}(s_t,a_t)=(1-\alpha)Q_t(s_t,a_t) + \alpha (r_t + \gamma \max_a Q(s_{t+1},a))$$
where $\alpha$ is a learning rate, and $\gamma$ is the discount factor.
The discount factor provides a weighting that places importance on rewards obtained sooner.

### Training

Let us implement training for the agent.
Some preliminaries: we denote a single session of the environment as an **episode**.
**Episodes** can be thought of a intantiations of a RL session (think of video games) that an agent "plays through" until it succeeds or fails to reach its objective.

We can start with a `q_table` with values of 0 for all state-action pairs: this is a naive agent with no idea of the rules and dynamics of this world.
The agent then starts interacting with the environment, choosing actions based on our $\epsilon$-greedy approach.
With each action the agent takes, it receives a reward which it can then use to *learn* more about the environment by updating the $Q$ table.
Over time, the agent should learn to pick successive actions that allow it to solve the `taxi-v3` environment.

In [6]:
import numpy as np
import random
from IPython.display import clear_output
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1
num_episodes = 100000

# For plotting metrics
all_steps = []
all_penalties = []

for i in range(num_episodes): # run for several episodes
    state = env.reset() # start with a random state

    steps, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        steps += 1
    
    all_steps.append(steps)
    
    if (i+1) % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i+1}: {np.mean(all_steps)} timesteps/episode")
        all_steps = []

print("Training finished.\n")

Episode: 100000: 14.55 timesteps/episode
Training finished.



In [7]:
sum(~np.all(q_table, axis=1))

100

### Evaluate agent's performance


In [8]:
total_steps, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset() # start with a random state
    steps, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        steps += 1

    total_penalties += penalties
    total_steps += steps

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_steps / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.8
Average penalties per episode: 0.0


### Animation

Here is an animation of a trained agent in action.

In [9]:
env.s = 328  # set environment to illustration's state

steps = 0
penalties, reward = 0, 0

frames_q = [] # for animation

done = False

while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames_q.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    steps += 1
    
    
print("Timesteps taken: {}".format(steps))
print("Penalties incurred: {}".format(penalties))


Timesteps taken: 12
Penalties incurred: 0


In [10]:
print_frames(frames_q, dt=1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 12
State: 0
Action: 5
Reward: 20


## References
1. https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/
2. https://gym.openai.com/envs/Taxi-v3/